## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [42]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
#TUNING NOTES: Attempted to omit INCOME_AMT, DID NOT WORK;
application_df_clean = application_df.drop(columns=["EIN"])


In [43]:
# I was attempting to get GPU acceleration for this project, and used this function to see if my GPU was being identified
physical_gpus = tf.config.list_physical_devices('GPU')
physical_gpus

[]

In [44]:
# Determine the number of unique values in each column.
for column in application_df_clean.columns:
    print(f"{column} has {application_df_clean[column].nunique()} unique values")

NAME has 19568 unique values
APPLICATION_TYPE has 17 unique values
AFFILIATION has 6 unique values
CLASSIFICATION has 71 unique values
USE_CASE has 5 unique values
ORGANIZATION has 4 unique values
STATUS has 2 unique values
INCOME_AMT has 9 unique values
SPECIAL_CONSIDERATIONS has 2 unique values
ASK_AMT has 8747 unique values
IS_SUCCESSFUL has 2 unique values


In [45]:
# Look at APPLICATION_TYPE value counts for binning
app_counts = application_df_clean["APPLICATION_TYPE"].value_counts()
print(app_counts)

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64


In [46]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
#TUNING NOTES: Attempted to double cutoff. Did not help.
cutoff = 500
application_types_to_replace = app_counts[app_counts < cutoff].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df_clean['APPLICATION_TYPE'] = application_df_clean['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df_clean['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [32]:
# Look at CLASSIFICATION value counts for binning
class_counts = application_df_clean["CLASSIFICATION"].value_counts()
class_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [48]:
# Look at CLASSIFICATION value counts for binning
name_counts = application_df_clean["NAME"].value_counts()
name_counts

NAME
PARENT BOOSTER USA INC                                                  1260
TOPS CLUB INC                                                            765
UNITED STATES BOWLING CONGRESS INC                                       700
WASHINGTON STATE UNIVERSITY                                              492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                          408
                                                                        ... 
ST LOUIS SLAM WOMENS FOOTBALL                                              1
AIESEC ALUMNI IBEROAMERICA CORP                                            1
WEALLBLEEDRED ORG INC                                                      1
AMERICAN SOCIETY FOR STANDARDS IN MEDIUMSHIP & PSYCHICAL INVESTIGATI       1
WATERHOUSE CHARITABLE TR                                                   1
Name: count, Length: 19568, dtype: int64

In [49]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_counts[class_counts > 1]

# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
#TUNING NOTES: Attempted to double cutoff. Did not help
cutoff = 1000
classifications_to_replace = class_counts[class_counts < cutoff].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df_clean['CLASSIFICATION'] = application_df_clean['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df_clean['CLASSIFICATION'].value_counts()


CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [50]:
# You may find it helpful to look at Name value counts >5
name_counts[name_counts > 1]

# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
#TUNING NOTES: Attempted to double cutoff. Did not help
cutoff = 5
names_to_replace = name_counts[name_counts < cutoff].index.tolist()

# Replace in dataframe
for cls in names_to_replace:
    application_df_clean['NAME'] = application_df_clean['NAME'].replace(cls,"Other")
    # application_df_clean['NAME'].value_counts().loc[lambda x:x>1]
    
# Check to make sure binning was successful
application_df_clean['NAME'].value_counts()

NAME
Other                                                                19803
PARENT BOOSTER USA INC                                                1260
TOPS CLUB INC                                                          765
UNITED STATES BOWLING CONGRESS INC                                     700
WASHINGTON STATE UNIVERSITY                                            492
                                                                     ...  
GUILDS OF THE SANTA FE OPERA INC                                         5
FAMILY MOTOR COACH ASSOCIATION                                           5
JAMESTOWN SOCIETY                                                        5
MOST WORSHIPFUL PRINCE HALL GRAND LODGE OF FREE & ACCEPTED MASONS        5
PTA HAWAII CONGRESS                                                      5
Name: count, Length: 403, dtype: int64

In [51]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_encoded = pd.get_dummies(application_df_clean, columns=["NAME","APPLICATION_TYPE", "AFFILIATION", "CLASSIFICATION", "USE_CASE", "ORGANIZATION", "SPECIAL_CONSIDERATIONS", "INCOME_AMT"],dtype=int)

In [52]:
# Split our preprocessed data into our features and target arrays
X = application_df_encoded.drop(columns=["IS_SUCCESSFUL"])
y = application_df_encoded["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)


In [ ]:
# Split our preprocessed data into our features and target arrays
X = application_df_encoded.drop(columns=["IS_SUCCESSFUL"])
y = application_df_encoded["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [ ]:
# Split our preprocessed data into our features and target arrays
X = application_df_encoded.drop(columns=["IS_SUCCESSFUL"])
y = application_df_encoded["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [ ]:
# Split our preprocessed data into our features and target arrays
X = application_df_encoded.drop(columns=["IS_SUCCESSFUL"])
y = application_df_encoded["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [53]:
X_train.head()

,STATUS,ASK_AMT,NAME_AACE INTERNATIONAL,NAME_ACE MENTOR PROGRAM OF AMERICA INC,NAME_ACTS MINISTRY,NAME_ACTS MISSIONS,NAME_AFRICAN-AMERICAN POSTAL LEAGUE UNITED FOR SUCCESS A-PLUS,NAME_AIR FORCE ASSOCIATION,NAME_ALABAMA FEDERATION OF WOMENS CLUBS,NAME_ALABAMA TREASURE FOREST ASSOCIATION,...,SPECIAL_CONSIDERATIONS_Y,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
30531,1,5000,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
22775,1,5000,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
19057,1,5000,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
27536,1,5000,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
21331,1,11121098,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [54]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [55]:
X_train_scaled[0:5]

array([[ 0.01394306, -0.03348439, -0.01527415, ..., -0.35186578,
        -0.06401969, -0.06987849],
       [ 0.01394306, -0.03348439, -0.01527415, ..., -0.35186578,
        -0.06401969, -0.06987849],
       [ 0.01394306, -0.03348439, -0.01527415, ..., -0.35186578,
        -0.06401969, -0.06987849],
       [ 0.01394306, -0.03348439, -0.01527415, ..., -0.35186578,
        -0.06401969, -0.06987849],
       [ 0.01394306,  0.10096552, -0.01527415, ..., -0.35186578,
        -0.06401969, -0.06987849]])

In [56]:
X_train_scaled.shape[1]

446

## Compile, Train and Evaluate the Model

In [57]:
# # Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=100, activation="relu", input_dim=X_train_scaled.shape[1]))

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=30, activation="sigmoid"))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=10, activation="sigmoid"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

c:\Users\Andrew Koller\anaconda3\envs\tf_new\lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_8 (Dense)                 │ (None, 100)            │        44,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 30)             │         3,030 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 10)             │           310 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 48,051 (187.70 KB)

 Trainable params: 48,051 (187.70 KB)

 Non-trainable params: 0 (0.00 B)

In [58]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [59]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 693us/step - accuracy: 0.7328 - loss: 0.5458
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 693us/step - accuracy: 0.7940 - loss: 0.4427
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 657us/step - accuracy: 0.7975 - loss: 0.4300
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 682us/step - accuracy: 0.7985 - loss: 0.4294
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 682us/step - accuracy: 0.7986 - loss: 0.4237
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 692us/step - accuracy: 0.7991 - loss: 0.4274
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 676us/step - accuracy: 0.8014 - loss: 0.4212
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 658us/step - accuracy: 0.8036 - loss: 0.4167
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 657us/step - accuracy: 0.8035 - loss: 0.4164
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 654us/step - accuracy: 0.8008 - loss: 0.4208
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 667us/step - accuracy: 0.7998 - loss: 0.4171
Epoch 12/100
804/80

In [60]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 ━━━━━━━━━━━━━━━━━━━━ 0s 487us/step - accuracy: 0.7869 - loss: 0.4487
Loss: 0.4486432671546936, Accuracy: 0.789970874786377


In [ ]:
# Export our model to HDF5 file
nn_model.save("AlphabetSoupCharity1-7258.h5")

## Tuning Workspace

In [ ]:
# This section ultimately proved fruitless, as the amount of time it would take to train a model with the 
#Model Tuning with Hyperparameters attempt 1
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=80,
        step=5), activation=activation, input_dim=X_train_scaled.shape[1]))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 2, 3)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=80,
            step=10),
            activation=activation,
            input_dim = X_train_scaled.shape[1]))
       
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model

In [ ]:
# Import the kerastuner library
import keras_tuner as kt
from keras.callbacks import Callback

class AccuracyThresholdStop(Callback):
    def __init__(self, threshold):
        super(AccuracyThresholdStop, self).__init__()
        self.threshold = threshold

    def on_trial_end(self, trial, model, epoch, logs=None):
        if logs['val_accuracy'] >= self.threshold:
            self.model.stop_training = True

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=15,
    hyperband_iterations=10,
    project_name = 'test_7')

stop_callback = AccuracyThresholdStop(threshold=0.75)

In [ ]:
tuner.search(X_train_scaled, y_train, epochs=20, validation_data=(X_test_scaled, y_test), callbacks=[stop_callback])

In [ ]:
top_hyper = tuner.get_best_hyperparameters(3)
for param in top_hyper:
    print(param.values)

In [ ]:
top_model = tuner.get_best_models(3)
for model in top_model:
    model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Parameters:{param.values} Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
#Tuning Attempt 2
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=X_train_scaled.shape[1]))
nn_model.add(tf.keras.layers.Dense(units=80, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=80, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Learning rates to try 
learning_rates = [0.1]

#iterating through the learning rates
for lr in learning_rates:
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    nn_model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    fit_model = nn_model.fit(X_train_scaled, y_train, epochs=200, verbose = 1)

    model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=1)
    print(f"Learning Rate: {lr} Loss: {model_loss}, Accuracy: {model_accuracy}")